In [1]:
# hide
# default_exp tests.core.test_compose
from nbdev.showdoc import *
from block_types.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Test compose

In [2]:
# export
import pytest
import os
import joblib
from IPython.display import display
import pandas as pd
import numpy as np
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.utils import Bunch
from sklearn.preprocessing import FunctionTransformer

from block_types.core.compose import *
from block_types.core.block_types import (Component, 
                                          PandasComponent)
from block_types.core.utils import PickleIO
from block_types.utils.utils import remove_previous_results
from block_types.core.block_types import Component, PickleSaverComponent

import block_types.config.bt_defaults as dflt
from block_types.utils.utils import check_last_part

In [3]:
#export
@pytest.fixture (name='column_transformer_data')
def column_transformer_data_fixture():
    return column_transformer_data()

@pytest.fixture (name='multi_split_data')
def multi_split_data_fixture():
    return multi_split_data()

## Parallel

### `find_last_fitted_model` example

In [4]:
# export
from block_types.utils.dummies import make_pipe_fit1

def test_pipeline_find_last_fitted_model_seq_others ():
    path_results = 'test_pipeline_find_last_fitted_model_seq_start'
    remove_previous_results (path_results=path_results)
    
    # pipelines
    pipe1 = make_pipe_fit1 ()
    X = np.array([1,2,3]).reshape(-1,1)
    r1 = pipe1.fit_apply (X)
    
    # case 1: component A
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    with pytest.raises (RuntimeError):
        r2 = pipe2.fit_apply (None)
    remove_previous_results (path_results=path_results)
    
    # case 2: component B
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)
    
    # case 2: component C
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    r = pipe2.C.fit_apply (r)
    all_fitted = pipe2.find_last_fitted_model ()
    assert not all_fitted
    pipe2.A.raise_error = True
    pipe2.B.raise_error = True
    pipe2.B.estimator = Bunch ()
    pipe2.C.estimator = Bunch ()
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)

In [5]:
tst.run (test_pipeline_find_last_fitted_model_seq_others, tag='dummy')

running test_pipeline_find_last_fitted_model_seq_others


fitting A (using whole data)
applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting pipeline (using whole data)
fitting A (using whole data)
fitting A (using whole data)
applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/A_result.pk
fitting B (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
applying B (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start/whole/B_result.pk
fitting pipeline (using whole data)
fitting B (using whole data)
loading from /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start/models/B_estimator.pk
loaded pre-trained B
applying B (on 

#### Second test case

In [14]:
# export
def test_pipeline_find_last_fitted_model_seq_others2 ():
    path_results = 'test_pipeline_find_last_fitted_model_seq_start2'
    remove_previous_results (path_results=path_results)
    
    # pipelines
    pipe1 = make_pipe_fit1 ()
    X = np.array([1,2,3]).reshape(-1,1)
    r1 = pipe1.fit_apply (X)
    
    # case 2: component C
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    r = pipe2.A.fit_apply (X)
    r = pipe2.B.fit_apply (r)
    r = pipe2.C.fit_apply (r)
    r = pipe2.D.fit_apply (r)
    r = pipe2.E.fit_apply (r)
    r = pipe2.F.fit_apply (r)
    r = pipe2.G.fit_apply (r)
    
    all_fitted = pipe2.find_last_fitted_model ()
    assert all_fitted
    pipe2.A.raise_error = True
    pipe2.B.raise_error = True
    pipe2.C.raise_error = True
    pipe2.D.raise_error = True
    pipe2.E.raise_error = True
    pipe2.F.raise_error = True
    #pipe2.G.raise_error = True
    
    pipe2.B.create_estimator ()
    pipe2.C.create_estimator ()
    pipe2.E.create_estimator ()
    pipe2.G.create_estimator ()
    pipe2.logger.info (f'\n{"-"*100}\n')
    
    r2 = pipe2.fit_apply (None)
    assert (r1==r2).all()
    remove_previous_results (path_results=path_results)
    
    # all
    pipe2 = make_pipe_fit1 (path_results=path_results, verbose=2)
    pipe2.logger.info (f'\n{"*"*100}\n')
    r2 = pipe2.fit_apply (X)
    
    all_fitted = pipe2.find_last_fitted_model ()
    assert all_fitted
    pipe2.A.raise_error = True
    pipe2.B.raise_error = True
    pipe2.C.raise_error = True
    pipe2.D.raise_error = True
    pipe2.E.raise_error = True
    pipe2.F.raise_error = True
    pipe2.G.raise_error = True
    
    pipe2.B.create_estimator ()
    pipe2.C.create_estimator ()
    pipe2.E.create_estimator ()
    pipe2.G.create_estimator ()
    
    pipe2.logger.info (f'\n{"-"*100}\n')
    r3 = pipe2.fit_apply (None)
    assert (r1==r3).all()
    remove_previous_results (path_results=path_results)

In [15]:
tst.run (test_pipeline_find_last_fitted_model_seq_others2, tag='dummy')

running test_pipeline_find_last_fitted_model_seq_others2


fitting A (using whole data)
applying A (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start2/whole/A_result.pk
fitting B (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start2/models/B_estimator.pk
applying B (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start2/whole/B_result.pk
fitting C (using whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start2/models/C_estimator.pk
applying C (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_start2/whole/C_result.pk
fitting D (using whole data)
applying D (on whole data)
saving to /home/jcidatascience/jaume/workspace/remote/block-types/test_pipeline_find_last_fitted_model_seq_s